#Predict what makes for a successful bike station

In [65]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

In [66]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (14.0, 6.0)

In [67]:
weather = pd.read_table('daily_weather.tsv')
usage = pd.read_table('usage_2012.tsv')
stations = pd.read_table('stations.tsv')

#Compute the number of rentals per station per day

In [68]:
days = usage[['station_start', 'time_start']]
days['day'] = days['time_start'].str[5:10]
days['count'] = 1

/Users/Scott/.virtualenvs/bike_share/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/Scott/.virtualenvs/bike_share/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


###Our stations data has a huge number of quantitative attributes: fast_food, parking, restaurant, etc... Some of them are encoded as 0 or 1 (for absence or presence), others represent counts. To start with, run a simple linear regression where the input (x) variables are all the various station attributes and the output (y) variable is the average number of rentals per day.

In [72]:
new_stations = pd.merge(days, stations, left_on='station_start', right_on='station')

In [61]:
new_stations.head()

,station_start,time_start,day,count,id,station,terminal_name,lat,long,no_bikes,...,museum,sculpture,hostel,picnic_site,tour_guide,attraction,landmark,motel,guest_house,gallery
0,7th & R St NW / Shaw Library,2012-01-01 00:04:00,01-01,1,145,7th & R St NW / Shaw Library,31245,38.912719,-77.022155,5,...,0,0,0,0,0,0,0,0,0,0
1,7th & R St NW / Shaw Library,2012-01-01 00:04:00,01-01,1,145,7th & R St NW / Shaw Library,31245,38.912719,-77.022155,5,...,0,0,0,0,0,0,0,0,0,0
2,7th & R St NW / Shaw Library,2012-01-01 01:55:00,01-01,1,145,7th & R St NW / Shaw Library,31245,38.912719,-77.022155,5,...,0,0,0,0,0,0,0,0,0,0
3,7th & R St NW / Shaw Library,2012-01-01 01:55:00,01-01,1,145,7th & R St NW / Shaw Library,31245,38.912719,-77.022155,5,...,0,0,0,0,0,0,0,0,0,0
4,7th & R St NW / Shaw Library,2012-01-01 12:40:00,01-01,1,145,7th & R St NW / Shaw Library,31245,38.912719,-77.022155,5,...,0,0,0,0,0,0,0,0,0,0


In [79]:
pd.pivot_table(new_stations, values='count', index=['station', 'day'], aggfunc='count')

station                     day  
10th & E St NW              07-12    20
                            07-13    39
                            07-14    48
                            07-15    39
                            07-16    25
                            07-17    35
                            07-18    17
                            07-19    18
                            07-20    27
                            07-21    38
                            07-22    50
                            07-23    47
                            07-24    32
                            07-25    40
                            07-26    32
                            07-27    37
                            07-28    42
                            07-29    51
                            07-30    40
                            07-31    31
                            08-01    51
                            08-02    53
                            08-03    40
                            08-04    34
      